In [1]:
import tensorflow as tf
import numpy as np
import os
import math

In [2]:
img_dir = '/home/chrisjan/project/training/catdog/images/train/'
logs_train_dir = '/home/chrisjan/project/training/catdog/models/train/'
logs_eval_dir = '/home/chrisjan/project/training/catdog/models/eval/'
eval_ratio = 0.2
img_width = 208
img_height = 208
batch_size = 8
capacity = 20000
n_class = 2
learning_rate = 0.0001
MAX_STEP = 6000

In [3]:
def group_train_eval(img_dir, eval_ratio):
    cats = []
    label_cats = []
    dogs = []
    label_dogs = []
    for file in os.listdir(img_dir):
        name = file.split(sep='.')
        if name[0]=='cat':
            cats.append(img_dir + file)
            label_cats.append(0)
        else:
            dogs.append(img_dir + file)
            label_dogs.append(1)
    #print('There are %d cats\nThere are %d dogs' %(len(cats), len(dogs)))
    image_list = np.hstack((cats, dogs))
    label_list = np.hstack((label_cats, label_dogs))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()
    np.random.shuffle(temp)
    all_image_list = temp[:, 0]
    all_label_list = temp[:, 1]
    n_sample = len(all_label_list)
    n_eval = math.ceil(n_sample * eval_ratio)
    n_train = n_sample - n_eval
    train_images = all_image_list[0:n_train]
    train_labels = all_label_list[0:n_train]
    train_labels = [int(float(i)) for i in train_labels]
    eval_images = all_image_list[n_train:-1]
    eval_labels = all_label_list[n_train:-1]
    eval_labels = [int(float(i)) for i in eval_labels]
    return train_images, train_labels, eval_images, eval_labels

def batch_train_eval(image, label, image_W, image_H, batch_size, capacity):
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
    input_queue = tf.train.slice_input_producer([image, label])
    label = input_queue[1]
    image_contents = tf.read_file(input_queue[0])
    image = tf.image.decode_jpeg(image_contents, channels=3)
    image = tf.image.resize_image_with_crop_or_pad(image, image_W, image_H)    
    image = tf.image.per_image_standardization(image)
    image_batch, label_batch = tf.train.batch([image, label], batch_size= batch_size, num_threads= 64, capacity = capacity)
    label_batch = tf.reshape(label_batch, [batch_size])
    image_batch = tf.cast(image_batch, tf.float32)
    return image_batch, label_batch

def inference(images, batch_size, n_classes):
    #conv1
    with tf.variable_scope('conv1') as scope:
        weights = tf.get_variable('weights', shape = [3,3,3,16], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.1,dtype=tf.float32))
        biases = tf.get_variable('biases', shape=[16], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(images, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name = scope.name)
    
    #pool1 and norm1   
    with tf.variable_scope('pooling1_lrn') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME', name='pooling1')
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75,name='norm1')
    
    #conv2
    with tf.variable_scope('conv2') as scope:
        weights = tf.get_variable('weights', shape=[3,3,16,16], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.1,dtype=tf.float32))
        biases = tf.get_variable('biases', shape=[16], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(norm1, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name='conv2')
    
    #pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75,name='norm2')
        pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME',name='pooling2')
    
    #local3
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = tf.get_variable('weights', shape=[dim,128], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.005,dtype=tf.float32))
        biases = tf.get_variable('biases', shape=[128], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)    
    
    #local4
    with tf.variable_scope('local4') as scope:
        weights = tf.get_variable('weights', shape=[128,128], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.005,dtype=tf.float32))
        biases = tf.get_variable('biases', shape=[128], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name='local4')
        
    # softmax
    with tf.variable_scope('softmax_linear') as scope:
        weights = tf.get_variable('softmax_linear', shape=[128, n_classes], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.005,dtype=tf.float32))
        biases = tf.get_variable('biases', shape=[n_classes], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')
    
    return softmax_linear

def losses(logits, labels):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        tf.summary.scalar(scope.name+'/loss', loss)
    return loss

def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op

def evaluation(logits, labels):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        tf.summary.scalar(scope.name + '/accuracy', accuracy)
    return accuracy

In [4]:
train_images, train_labels, eval_images, eval_labels = group_train_eval(img_dir, eval_ratio)
train_batch, train_label_batch = batch_train_eval(train_images, train_labels, img_width, img_height, batch_size, capacity)
eval_batch, eval_label_batch = batch_train_eval(eval_images, eval_labels, img_width, img_height, batch_size, capacity)

In [5]:
logits = inference(train_batch, batch_size, n_class)
loss = losses(logits, train_label_batch)
train_op = trainning(loss, learning_rate)
acc = evaluation(logits, train_label_batch)
x = tf.placeholder(tf.float32, shape=[batch_size, img_width, img_height, 3])
y_ = tf.placeholder(tf.int16, shape=[batch_size])

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    summary_op = tf.summary.merge_all()        
    train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
    eval_writer = tf.summary.FileWriter(logs_eval_dir, sess.graph)

    try:
        for step in np.arange(MAX_STEP):
            if coord.should_stop():
                break

            tra_images, tra_labels = sess.run([train_batch, train_label_batch])
            _, tra_loss, tra_acc = sess.run([train_op, loss, acc], feed_dict={x:tra_images, y_:tra_labels})
            if step % 50 == 0:
                print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
                summary_str = sess.run(summary_op)
                train_writer.add_summary(summary_str, step)

            if step % 200 == 0 or (step + 1) == MAX_STEP:
                eval_images, eval_labels = sess.run([eval_batch, eval_label_batch])
                val_loss, val_acc = sess.run([loss, acc], feed_dict={x:eval_images, y_:eval_labels})
                print('**  Step %d, val loss = %.2f, val accuracy = %.2f%%  **' %(step, val_loss, val_acc*100.0))
                summary_str = sess.run(summary_op)
                eval_writer.add_summary(summary_str, step)  

            if step % 2000 == 0 or (step + 1) == MAX_STEP:
                checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                saver.save(sess, checkpoint_path, global_step=step)

    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')
    finally:
        coord.request_stop()           
    coord.join(threads)

Step 0, train loss = 0.69, train accuracy = 43.75%
**  Step 0, val loss = 0.69, val accuracy = 43.75%  **
Step 50, train loss = 0.67, train accuracy = 62.50%
Step 100, train loss = 0.70, train accuracy = 43.75%
Step 150, train loss = 0.69, train accuracy = 56.25%
